# O PROBLEMA

Imagine agora que você foi contratado(a) como Expert em Data Analytics por um grande hospital para entender como foi o comportamento da população na época da pandemia da COVID-19 e quais indicadores seriam importantes para o planejamento, caso haja um novo surto da doença.

Apesar de ser contratado(a) agora, a sua área observou que a utilização do estudo do PNAD-COVID 19 do IBGE seria uma ótima base para termos boas respostas ao problema proposto, pois são dados confiáveis. Porém, não será necessário utilizar todas as perguntas realizadas na pesquisa para enxergar todas as oportunidades ali postas.

É sempre bom ressaltar que há dados triviais que precisam estar no projeto, pois auxiliam muito na análise dos dados:

## PNAD-COVID-19 do IBGE

O Head de Dados pediu para que você entrasse na base de dados do PNAD-COVID-19 do IBGE e organizasse esta base para análise, utilizando Banco de Dados em Nuvem e trazendo as seguintes características:

- a. Utilização de no máximo 20 questionamentos realizados na pesquisa;
- b. Utilizar 3 meses para construção da solução;
- c. Caracterização dos sintomas clínicos da população;
- d. Comportamento da população na época da COVID-19;
- e. Características econômicas da Sociedade;

Seu objetivo será trazer uma breve análise dessas informações, como foi a organização do banco, as perguntas selecionadas para a resposta do problema e quais seriam as principais ações que o hospital deverá tomar em caso de um novo surto de COVID-19.

- Dica: Leiam com atenção a base de dados e toda a documentação que o site o PNAD – Covid19 traz, principalmente os dicionários, que ajudam e muito no entendimento da Base de Dados.
- Dica 2: Utilizem o que já foi ensinado e consolidado nas outras fases para apresentar a resolução do projeto.

Lembre-se de que você poderá apresentar o desenvolvimento do seu projeto durante as lives com docentes. Essa é uma boa oportunidade para discutir sobre as dificuldades encontradas e pegar dicas valiosas com especialistas e colegas de turma.

Link para a base: https://www.ibge.gov.br/estatisticas/investigacoes-experimentais/estatisticas-experimentais/27946-divulgacao-semanal-pnadcovid1?t=downloads&utm_source=covid19&utm_medium=hotsite&utm_campaign=covid_19

### Importando bibliotecas

In [264]:
import warnings 
warnings.filterwarnings(action = 'ignore')

In [265]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from pyspark.sql import SparkSession
import findspark
import os
import pandas as pd
import polars as pl
import plotly.express as px
import plotly.graph_objects as go

### Conectando ao MySQL

In [266]:
'''def mysql_connection(host, user, passwd, database=None):
    engine = create_engine(f'mysql+pymysql://{user}:{passwd}@{host}/{database}')
    return engine.connect()

mysql = mysql_connection('127.0.0.1', 'root', 'admin', 'pnad_covid')'''

"def mysql_connection(host, user, passwd, database=None):\n    engine = create_engine(f'mysql+pymysql://{user}:{passwd}@{host}/{database}')\n    return engine.connect()\n\nmysql = mysql_connection('127.0.0.1', 'root', 'admin', 'pnad_covid')"

In [267]:
'''statement = text('SELECT * FROM pnad_covid_view LIMIT 5')
response = mysql.execute(statement)
for row in response:
        print(row)'''

"statement = text('SELECT * FROM pnad_covid_view LIMIT 5')\nresponse = mysql.execute(statement)\nfor row in response:\n        print(row)"

### Criando sessão Spark

In [268]:
os.environ['SPARK_HOME'] = '/opt/spark/'
findspark.init()

In [269]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [270]:
spark

### Importando dados

In [271]:
'''res = mysql.execute(text('SELECT * FROM pnad_covid_view LIMIT 1149196'))
res = res.fetchall()'''

"res = mysql.execute(text('SELECT * FROM pnad_covid_view LIMIT 1149196'))\nres = res.fetchall()"

In [272]:
'''df = pd.DataFrame(res)'''

'df = pd.DataFrame(res)'

In [273]:
'''df.to_csv('dados/dados_exportados/2023-10-25_pnad_covid_view.csv', index=False)'''

"df.to_csv('dados/dados_exportados/2023-10-25_pnad_covid_view.csv', index=False)"

### Dicionários

In [274]:
estados = {
    'Acre': 'AC',
    'Alagoas': 'AL',
    'Amapá': 'AP',
    'Amazonas': 'AM',
    'Bahia': 'BA',
    'Ceará': 'CE', 
    'Distrito Federal': 'DF',
    'Espírito Santo': 'ES',
    'Goiás': 'GO',
    'Maranhão': 'MA',
    'Mato Grosso': 'MT',
    'Mato Grosso do Sul': 'MS',
    'Minas Gerais': 'MG',
    'Pará': 'PA',
    'Paraíba': 'PB',
    'Paraná': 'PR',
    'Pernambuco': 'PE',
    'Piauí': 'PI', 
    'Rio de Janeiro': 'RJ',
    'Rio Grande do Norte': 'RN',
    'Rio Grande do Sul': 'RS',
    'Rondônia': 'RO',
    'Roraima': 'RR',
    'Santa Catarina': 'SC',
    'São Paulo': 'SP',
    'Sergipe': 'SE',
    'Tocantins': 'TO'
}

regioes = {
'AC': 'Norte',
'AL': 'Nordeste',
'AP': 'Norte',
'AM': 'Norte',
'BA': 'Nordeste',
'CE': 'Nordeste',
'DF': 'Centro-Oeste',
'ES': 'Sudeste',
'GO': 'Centro-Oeste',
'MA': 'Nordeste',
'MT': 'Centro-Oeste',
'MS': 'Centro-Oeste', 
'MG': 'Sudeste',
'PA': 'Norte',
'PB': 'Nordeste',
'PR': 'Sul',
'PE': 'Nordeste',
'PI': 'Nordeste',
'RJ': 'Sudeste',
'RN': 'Nordeste',
'RS': 'Sul',
'RO': 'Norte',
'RR': 'Norte',
'SC': 'Sul',
'SP': 'Sudeste',
'SE': 'Nordeste',
'TO': 'Norte'
}

### Análise exploratória de dados econômicos

In [275]:
df = spark.read.csv('dados/dados_exportados/2023-10-25_pnad_covid_view.csv', sep = ',', inferSchema = True, header = True)


In [276]:
df.show(5)

+----------+--------------+------------------+-----+------+--------+--------------------+-----------------------------+-----------------------+-----------------------------+---------------------------------+--------------------+------------------------+------------------+---------------------------------+--------------------------+---------------------------+-------------------------------+----------------+-------------------------------+------------------------+---------------------+-----------------------+-----------------+--------------------+--------------------+-------------+-----------------------+-----------+---------------------------+-------------+-----------------+---------------+
|      data|            uf|situacao_domicilio|idade|  sexo|cor_raca|        escolaridade|questao_estabelecimento_saude|questao_permaneceu_casa|questao_remedio_conta_propria|questao_remedio_orientacao_medica|questao_hospital_SUS|questao_hospital_privado|questao_internacao|questao_internacao_ajuda_res

In [277]:
df.printSchema()

root
 |-- data: date (nullable = true)
 |-- uf: string (nullable = true)
 |-- situacao_domicilio: string (nullable = true)
 |-- idade: integer (nullable = true)
 |-- sexo: string (nullable = true)
 |-- cor_raca: string (nullable = true)
 |-- escolaridade: string (nullable = true)
 |-- questao_estabelecimento_saude: string (nullable = true)
 |-- questao_permaneceu_casa: string (nullable = true)
 |-- questao_remedio_conta_propria: string (nullable = true)
 |-- questao_remedio_orientacao_medica: string (nullable = true)
 |-- questao_hospital_SUS: string (nullable = true)
 |-- questao_hospital_privado: string (nullable = true)
 |-- questao_internacao: string (nullable = true)
 |-- questao_internacao_ajuda_respirar: string (nullable = true)
 |-- questao_motivo_afastamento: string (nullable = true)
 |-- questao_tempo_afastado_trab: string (nullable = true)
 |-- questao_tipo_trabalho_realizado: string (nullable = true)
 |-- faixa_rendimento: string (nullable = true)
 |-- rendimento_aposentado

#### Adicionando colunas

In [278]:
df = df.withColumn('sigla', df.uf).replace(to_replace=estados, subset=['sigla']) #adicionando sigla dos estados
df = df.withColumn('regiao', df.sigla).replace(to_replace=regioes, subset=['regiao']) #adicionando regioes dos estados
df.show(5)

+----------+--------------+------------------+-----+------+--------+--------------------+-----------------------------+-----------------------+-----------------------------+---------------------------------+--------------------+------------------------+------------------+---------------------------------+--------------------------+---------------------------+-------------------------------+----------------+-------------------------------+------------------------+---------------------+-----------------------+-----------------+--------------------+--------------------+-------------+-----------------------+-----------+---------------------------+-------------+-----------------+---------------+-----+--------+
|      data|            uf|situacao_domicilio|idade|  sexo|cor_raca|        escolaridade|questao_estabelecimento_saude|questao_permaneceu_casa|questao_remedio_conta_propria|questao_remedio_orientacao_medica|questao_hospital_SUS|questao_hospital_privado|questao_internacao|questao_inter

In [279]:
df_temp = df.createOrReplaceTempView('df_temp') #criando tabela temporária na sessão Spark SQL

#### Distribuição de infectados por zona

In [303]:
df_infectados_por_zona = spark.sql(
    '''
        SELECT situacao_domicilio, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY situacao_domicilio
        ORDER BY situacao_domicilio
    '''
).toPandas()

fig = px.bar(
    df_infectados_por_zona.qtd_infectados,
    color = df_infectados_por_zona.situacao_domicilio
)

fig.update_layout(
    width=800, 
    height=600
)

fig.show()

#### Distribuição de infectados por sexo

In [308]:
df_infectados_por_sexo = spark.sql(
    '''
        SELECT sexo, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY sexo
        ORDER BY sexo
    '''
).toPandas()

fig = px.bar(
    df_infectados_por_sexo.qtd_infectados,
    color = df_infectados_por_sexo.sexo
)

fig.update_layout(
    width=800, 
    height=600
)

fig.show()

#### Distribuição de infectados por cor/raça

In [317]:
df_infectados_por_sexo

,sexo,cor_raca,qtd_infectados
0,Homem,Ignorado,10
1,Homem,Branca,5605
2,Homem,Preta,1292
3,Homem,Amarela,51
4,Homem,Indígena,98
5,Homem,Parda,7276
6,Mulher,Indígena,146
7,Mulher,Amarela,83
8,Mulher,Branca,7193
9,Mulher,Parda,8902


In [329]:
df_infectados_por_cor_raca = spark.sql(
    '''
        SELECT cor_raca, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY cor_raca
        ORDER BY cor_raca
    '''
).toPandas()

fig = px.bar(
    df_infectados_por_cor_raca.qtd_infectados,
    color = df_infectados_por_cor_raca.cor_raca
)

fig.update_layout(
    width=800, 
    height=600
)

fig.show()

#### Distribuição de infectados por escolaridade

In [309]:
df_infectados_por_escolaridade = spark.sql(
    '''
        SELECT escolaridade, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY escolaridade
        ORDER BY escolaridade
    '''
).toPandas()

fig = px.bar(
    df_infectados_por_escolaridade.qtd_infectados,
    color = df_infectados_por_escolaridade.escolaridade
)

fig.update_layout(
    width=800, 
    height=600
)

fig.show()

#### Contagio por estado

In [307]:
df_cotagio_por_estado = spark.sql(
    '''
        SELECT uf, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY uf
        ORDER BY uf
    '''
).toPandas()

fig = px.bar(
    df_cotagio_por_estado.qtd_infectados,
    color = df_cotagio_por_estado.uf
)

fig.update_layout(
    width=800, 
    height=600
)

fig.show()

#### Contagio por região

In [281]:
df_cotagio_por_regiao = spark.sql(
    '''
        SELECT regiao, count(resultado_teste) as qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY regiao
        ORDER BY regiao
    '''
).toPandas()

fig = px.bar(
    df_cotagio_por_regiao.qtd_infectados,
    color = df_cotagio_por_regiao.regiao
)

fig.update_layout(
    width=800, 
    height=600
)

fig.show()

#### Taxa de contagio por estado

In [282]:
df_taxa_de_contagio_por_estado = spark.sql(
    '''
        SELECT uf, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY uf
        ORDER BY uf
    '''
).toPandas()

df_qtd_testes_validos_por_estado = spark.sql(
    '''
        SELECT uf, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY uf
        ORDER BY uf
    '''
).toPandas()

df_taxa_de_contagio_por_estado['qtd_testes_validos'] = df_qtd_testes_validos_por_estado['qtd_testes_validos']
df_taxa_de_contagio_por_estado['taxa_de_contagio_por_mil_habitantes'] = ((df_taxa_de_contagio_por_estado['qtd_infectados'] / df_qtd_testes_validos_por_estado['qtd_testes_validos']) * 1000).round().astype(int)

fig = px.bar(
    df_taxa_de_contagio_por_estado.taxa_de_contagio_por_mil_habitantes,
    color = df_taxa_de_contagio_por_estado.uf
)

fig.update_layout(
    width=800, 
    height=600
)

fig.show()

#### Taxa de contagio por região

In [283]:
df_taxa_de_contagio_por_regiao = spark.sql(
    '''
        SELECT regiao, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY regiao
        ORDER BY regiao
    '''
).toPandas()

df_qtd_testes_validos_por_regiao = spark.sql(
    '''
        SELECT regiao, count(teste_covid) AS qtd_testes_validos
        FROM df_temp 
        WHERE teste_covid = 'Sim' AND (resultado_teste = 'Positivo' OR resultado_teste = 'Negativo')
        GROUP BY regiao
        ORDER BY regiao
    '''
).toPandas()

df_taxa_de_contagio_por_regiao['qtd_testes_validos'] = df_qtd_testes_validos_por_regiao['qtd_testes_validos']
df_taxa_de_contagio_por_regiao['taxa_de_contagio_por_mil_habitantes'] = ((df_taxa_de_contagio_por_regiao['qtd_infectados'] / df_qtd_testes_validos_por_regiao['qtd_testes_validos']) * 1000).round().astype(int)

fig = px.bar(
    df_taxa_de_contagio_por_regiao.taxa_de_contagio_por_mil_habitantes,
    color = df_taxa_de_contagio_por_regiao.regiao
)

fig.update_layout(
    width=800, 
    height=600
)

fig.show()

#### Taxa de contagio por idade nos estados

In [284]:
df_cotagio_por_faixa_etaria = spark.sql(
    '''
        SELECT
                count(resultado_teste) AS qtd_infectados,
                (CASE
                    WHEN idade BETWEEN  1 AND 4 THEN '6 meses a 4 anos de idade'
                    WHEN idade BETWEEN 3 AND 4 THEN '3 e 4 anos de idade'
                    WHEN idade BETWEEN 5 AND 11 THEN '5 a 11 anos de idade'
                    WHEN idade BETWEEN 12 AND 39 THEN '12 a 39 anos de idade'
                    WHEN idade BETWEEN 40 AND 59 THEN '40 a 59 anos de idade'
                    WHEN idade >= 60 THEN 'Mais de 60 anos de idade'
                    ELSE 'Idade não informada' 
                END) AS faixa_etaria
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY faixa_etaria
        ORDER BY faixa_etaria
    '''
).toPandas()


fig = px.bar(
    df_cotagio_por_faixa_etaria.qtd_infectados,
    color = df_cotagio_por_faixa_etaria.faixa_etaria
)

fig.update_layout(
    width=800, 
    height=600
)

fig.show()

#### Distribuição da renda média de infectados

In [335]:
df_faixa_rendimento_infectados = spark.sql(
    '''
        SELECT  
                faixa_rendimento, 
                count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY faixa_rendimento
        ORDER BY faixa_rendimento
    '''
).toPandas()

fig = px.bar(
    df_faixa_rendimento_infectados.qtd_infectados,
    color = df_faixa_rendimento_infectados.faixa_rendimento
)

fig.update_layout(
    width=800, 
    height=600
)

fig.show()

#### Distribuição de métodos de testagem

In [336]:
df_tipo_teste_covid = spark.sql(
    '''
        SELECT tipo_teste, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo'
        GROUP BY tipo_teste
        ORDER BY tipo_teste
    '''
).toPandas()

fig = px.bar(
    df_tipo_teste_covid.qtd_infectados,
    color = df_tipo_teste_covid.tipo_teste
)

fig.update_layout(
    width=800, 
    height=600
)

fig.show()

In [337]:
df.show()

+----------+-----------------+------------------+-----+------+--------+--------------------+-----------------------------+-----------------------+-----------------------------+---------------------------------+--------------------+------------------------+------------------+---------------------------------+--------------------------+---------------------------+-------------------------------+----------------+-------------------------------+------------------------+---------------------+-----------------------+-----------------+--------------------+--------------------+-------------+-----------------------+-----------+---------------------------+--------------------+-----------------+---------------+-----+--------+
|      data|               uf|situacao_domicilio|idade|  sexo|cor_raca|        escolaridade|questao_estabelecimento_saude|questao_permaneceu_casa|questao_remedio_conta_propria|questao_remedio_orientacao_medica|questao_hospital_SUS|questao_hospital_privado|questao_internacao|

#### Que áreas profissionais foram mais afetadas pelo contagio no período?

In [351]:
df_tipo_trabalho = spark.sql(
    '''
        SELECT questao_tipo_trabalho_realizado, count(resultado_teste) AS qtd_infectados
        FROM df_temp 
        WHERE resultado_teste = 'Positivo' AND questao_tipo_trabalho_realizado != 'Não aplicável' AND questao_tipo_trabalho_realizado != 'Outros'
        GROUP BY questao_tipo_trabalho_realizado
        ORDER BY qtd_infectados, questao_tipo_trabalho_realizado
    '''
).toPandas()

fig = px.bar(
    df_tipo_trabalho,
    x = df_tipo_trabalho.qtd_infectados,
    color = df_tipo_trabalho.questao_tipo_trabalho_realizado
)

fig.update_layout(
    width=1200, 
    height=800
)

fig.show()